In [1]:
#load up the basics
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
ModelPredFile = '../data/tables/MasterDF_09102021b.csv'
preds = pd.read_csv(ModelPredFile)
preds.head()
print(preds.shape)

(135245, 21)


In [3]:
#preds = preds[~preds['Filename'].str.contains(str('20180916b_jpgs/jpgs/bak'))]
#preds.shape

(135245, 21)

In [5]:
#THIS NEEDS TO EXCLUDE PREIOUSLY LABELED IMAGES FOR R6
#AND ALSO NEEDS TO MARK IMAGES AS LABELED in the dataframe and then save a new copy

In [4]:
def Picker(Storm,roundNumber,digit,df):
    
    ######FIND THE IMAGES THAT ARE UNCERTAIN
    
    #select the storm
    StormSubset = df[df['Filename'].str.contains(str(Storm))]
    
    prediction = "Prediction" + str(digit - 1)
    
    #make new column certainty of prediction, from 0 to 0.5... 0 is smallest certainty
    StormSubset["certainty"] = abs(StormSubset[str(prediction)] - 0.5)
    
    #find the 100 that have not been labeled. change to 50 for round 6
    smallest = StormSubset[StormSubset.labeled.isna()].nsmallest(50, 'certainty')
    

    ######MARK THE IMAGES THAT ARE UNCERTAIN  
    
    #add labeled attribute to the dataframe, to denote that this image is slected to be labeled 
    #(number denotes the round that causes the image to be selected.
    smallest['Labeled'] = (digit)
    
    smolSubset = smallest[["Filename","Labeled"]]
    
    #merge to prediction dataframe
    df = pd.merge(df, smolSubset, on="Filename", how = "left")
    
    #combine Labeled and labeled
    df['labeled'].fillna(df['Labeled'], inplace=True)
    df = df.drop(columns=['Labeled'])
    
    ###### MOVE THE IMAGES THAT ARE UNCERTAIN
    
    #make a new dir
    newdir = '../data/pics/modeltoLabeler/' + str(roundNumber) + '/' + str(Storm) + str(digit)
    os.mkdir(newdir)
    
    smallest['Image_Path'] = '/data/NOAA_ERI/' + smallest['Filename'].astype(str)
    
    #pull out the list from the df
    MoveList= smallest['Image_Path'].tolist()

    #move those on the list
    for f in MoveList:
        shutil.copy(f, newdir)
        
    #print(smallest.head())   
    
    return df


In [7]:
#Test = Picker('Barry','roundSix', 6, preds)
#Test['labeled'].max()

In [5]:
df = Picker('Barry','roundSeven', 7, preds)
df = Picker('Delta','roundSeven', 7, df)
df = Picker('Gordon','roundSeven', 7, df)
df = Picker('Laura','roundSeven', 7, df)
df = Picker('Sally','roundSeven', 7, df)
df = Picker('Zeta','roundSeven', 7, df)

#df = Picker('Henri','roundSix', 6, df)
#df = Picker('Ida','roundSix', 6, df)

#df = Picker('Dorian','roundSix', 6, df)
#df = Picker('Isaias','roundSix', 6, df)
#df = Picker('Michael','roundSix', 6, df)
#Florence has duplicate image nnames.. need to deal with that
#df = Picker('Florence','roundSix', 6, df)

<ipython-input-4-bc25e2a1f877>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  StormSubset["certainty"] = abs(StormSubset[str(prediction)] - 0.5)


In [6]:
df.head()

,Filename,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,archive,date,image,labeled,...,ll_lon,lr_lat,lr_lon,size,storm_id,ul_lat,ul_lon,ur_lat,ur_lon,Prediction6
0,Barry/20190716a_jpgs/jpgs/C19748935.jpg,0.093053,0.458411,0.679300,0.274590,0.021257,20190716a_jpgs,2019-07-16,C19748935.jpg,NaN,...,-92.321909,29.501893,-92.311318,4875221,barry,29.514684,-92.317575,29.509719,-92.306144,0.210064
1,Barry/20190716a_jpgs/jpgs/C19748941.jpg,0.002945,0.449371,0.456580,0.237515,0.043771,20190716a_jpgs,2019-07-16,C19748941.jpg,NaN,...,-92.319636,29.506580,-92.308422,5842249,barry,29.518644,-92.316049,29.514679,-92.304272,0.144417
2,Barry/20190716a_jpgs/jpgs/C19748947.jpg,0.002490,0.413135,0.469627,0.226322,0.052162,20190716a_jpgs,2019-07-16,C19748947.jpg,NaN,...,-92.317814,29.511428,-92.306184,6061687,barry,29.522755,-92.314927,29.519666,-92.302932,0.151956
3,Barry/20190716a_jpgs/jpgs/C19748953.jpg,0.020509,0.361904,0.444304,0.202061,0.065795,20190716a_jpgs,2019-07-16,C19748953.jpg,NaN,...,-92.316963,29.516382,-92.305242,5873595,barry,29.527209,-92.314396,29.524401,-92.302525,0.156361
4,Barry/20190716a_jpgs/jpgs/C19748959.jpg,0.043591,0.347595,0.443178,0.191845,0.061327,20190716a_jpgs,2019-07-16,C19748959.jpg,NaN,...,-92.315955,29.521040,-92.304028,6159050,barry,29.531420,-92.313768,29.529043,-92.301852,0.156329


In [7]:
#save the data

#new csv to save
newpredcsv = '../data/tables/MasterDF_09102021b_R7.csv'

df.to_csv(newpredcsv, index=False)

In [ ]:
#looking at duplicate florence names
#Fdf = df.loc[(df['storm_id'] == 'florence') & (df['labeled'] ==6)]
#len(Fdf['Filename'].drop_duplicates())